# Setup do Código

Aqui o espaço para setar todos as variáveis para importar planilha

In [1]:
# Colocar Caminho da Pasta, que armazena os Excels de Consulta (usar '/', para o caminho)
PATH = './src'
CLIENT_PATH = './src/Clientes'

# Inserir Nome do Excel, com a extensão
portfolioCliente = 'Portfolio Global - KING.xlsx'

databaseFundos = 'New_query_2023_11_16 (1).xlsx'
historicoBenchmarks = 'arvore_classificacao_insper.xlsx'


# Limite de Tracking Error de Acordo com o Perfil de Risco do Investidor
perfilRisco = 1.1 # em porcentagem [%]

# Quantidade de Aplicações que vão ser Rebalanceadas
nAplicacoes = 4

<br>

---

<br>

# Rebalanceamento de Carteira

## Importando Bibliotecas

Bibliotecas Importantes usadas para o Código
- OS: pode criar pastas e arquivos no sistema
- Math: Usada para cálculos
- Locale: Importar dados de formatação por região de números
- Numpy: Processamento de Matrizes e Arrays
- Pandas: Manipulação de Dataframes, e planilhas de Excel
- Openpyxl: Leitura e Escrita de Excels
- Scikit-Image (skimage): Processamento de Imagens (processamento de planilhas)
- Unidecode: Decodar certos Carácteres
- Scipy (scipy.optimize): Serve para Minimar a Funcão Objetivo
- Plyer (notification): Manda Notificação pelo Computador

In [2]:
import os
import math
import locale
import numpy as np
import pandas as pd
import openpyxl as xl 
from skimage.measure import label, regionprops
from unidecode import unidecode
from scipy.optimize import minimize
from plyer import notification

# Setups das Bibliotecas
pd.options.mode.chained_assignment = None  # default='warn'
locale.setlocale(locale.LC_ALL, 'pt-BR.utf-8')


'pt-BR.utf-8'

## Leitura dos Excels e Planilhas:

Função para ler e receber os nomes de cada planilha dentro do excel

In [3]:
# Recebe Caminho do Arquivo, e Nome do Arquivo

def getSheetNames(path, file):
    excel_file = f'{path}/{file}'
    wb = xl.load_workbook(excel_file)
    
    if len(wb.sheetnames) == 1:
        return [wb.sheetnames[0]]
    else:
        return wb.sheetnames


In [4]:
# Carteira Portfólio Cliente
nameClient = getSheetNames(CLIENT_PATH, portfolioCliente)
substrings_to_check = ['carteira', 'política', 'politica']
nameClient = [name for name in nameClient if any(substring in name.lower() for substring in substrings_to_check)]
clientCarteira, clientPolitica = nameClient[0], nameClient[1]

# Base de Dados dos Fundos
nameResult = getSheetNames(PATH, databaseFundos)
resultBenchmark = nameResult[0]

# Árvore Classificação Insper
nameClass = getSheetNames(PATH, historicoBenchmarks)
classBenchmarks, classRetornos = nameClass[0], nameClass[1]

Leitura dos Excels da Empresa

In [5]:
# Carteira do Cliente:
carteira = pd.read_excel(f'{CLIENT_PATH}/{portfolioCliente}', sheet_name=clientCarteira, header=None)
politica = pd.read_excel(f'{CLIENT_PATH}/{portfolioCliente}', sheet_name=clientPolitica, header=None)

# Base de Dados Fundos
database = pd.read_excel(f'{PATH}/{databaseFundos}', sheet_name=resultBenchmark, header=None)

# Árvore Classificação Insper
benchmarks = pd.read_excel(f'{PATH}/{historicoBenchmarks}', sheet_name=classBenchmarks, header=None)
retornosBenchmarks = pd.read_excel(f'{PATH}/{historicoBenchmarks}', sheet_name=classRetornos, header=None)

## Extração de Tabelas e Atribuição de Variáveis Dataframe

#### Extração de Tabelas dentro de cada Excel:

In [6]:
# Função para Ler as Tabelas dentro de cada Planilha
def getTables(df):
    # Transformando Dataframe em 0s e 1s
    larr = label(np.array(df.notnull()).astype("int"))
    
    tables = []
    for s in regionprops(larr):
        # Filtro de "Tabelas" Pequenas
        ux, lx, uy, ly = s.bbox
        diff_u, diff_l = uy - ux, ly - lx
        if diff_u <= 1 or diff_l <= 1:
            continue
        
        # Criação do Sub-Dataframe 
        table = df.iloc[
            s.bbox[0]:s.bbox[2], s.bbox[1]:s.bbox[3]
        ].pipe(
            lambda df_: df_.rename(columns=df_.iloc[0]).drop(df_.index[0])
        )

        # Adicionando Sub-Dataframe a Lista de Tables
        tables.append(table)
        
    return tables


In [7]:
dfsCarteira = getTables(carteira)
dfsPolitica = getTables(politica)
dfsDatabase = getTables(database)
dfsBenchmarks = getTables(benchmarks)
dfsRetornos = getTables(retornosBenchmarks)

#### Criando Variáveis para cada Tabela dos Excels

In [8]:
def attrDataframes(list_dfs):
    def getVarName(df):
        def namestr(obj, namespace):
            return [name for name in namespace if namespace[name] is obj]
    
        return namestr(df, globals())[0]
    
    for i in range(len(list_dfs)):
        dfName = getVarName(list_dfs).replace('dfs', '')
        
        varName = f'df{dfName}{i+1}'
        value = list_dfs[i]
        globals()[varName] = value
        
        print(f'- Dataframe {varName} created.')
    print('')


In [9]:
attrDataframes(dfsCarteira)
attrDataframes(dfsPolitica)
attrDataframes(dfsDatabase)
attrDataframes(dfsBenchmarks)
attrDataframes(dfsRetornos)

- Dataframe dfCarteira1 created.

- Dataframe dfPolitica1 created.
- Dataframe dfPolitica2 created.
- Dataframe dfPolitica3 created.

- Dataframe dfDatabase1 created.

- Dataframe dfBenchmarks1 created.

- Dataframe dfRetornos1 created.



## Dataframes

### - Carteira do Cliente:

In [10]:
valorCarteira = dfCarteira1["Financeiro"].iloc[-1]
perctCarteira = dfCarteira1["%"].iloc[-1]

In [11]:
tradingDesk = 'Carteira'
carteira = 'Financeiro'

dfCarteira1 = dfCarteira1.dropna(subset=[tradingDesk])
dfCarteira1 = dfCarteira1.dropna(subset=[carteira])

In [12]:
dfCarteira1.head()

,Data Posição,Source,Carteira,Quantidade,Financeiro,Classe,%
4,NaN,NaN,Renda Fixa Brasil Pós Fixado,NaN,2871779.418973,NaN,0.233557
5,2023-09-13 00:00:00,KING - BRL,BV CASH FIRF SIMPLES,21170.299483,26943.031989,Renda Fixa Brasil Pós Fixado,0.002191
6,2023-09-13 00:00:00,KING - BRL,BTG PACTUAL DIGITAL TESOURO SELIC SIMPLES FI RF,218395.712765,315562.17117,Renda Fixa Brasil Pós Fixado,0.025664
7,2023-09-13 00:00:00,KING - BRL,LCI 30Jun2023 - 20Jun2025 96.5 BRADESCO,800000,820257.249879,Renda Fixa Brasil Pós Fixado,0.06671
8,2023-09-13 00:00:00,KING - BRL,LCA 30Aug2023 - 19Aug2025 94.5 BRADESCO,1000000,1004178.335827,Renda Fixa Brasil Pós Fixado,0.081668


### - Política de Investimento

In [13]:
dfPolitica1.head()

,TradingDesk,Book,Mínimo,Máximo,Estratégico,Tático,Atual
1,KING - BRL,Renda Fixa Brasil CDI,0.05,0.35,0.075,0.13,0.233557
2,KING - BRL,Renda Fixa Brasil Crédito Pós-Fixado,0,0.2,0.09,0.09,0.100853
3,KING - BRL,Renda Fixa Brasil Pré-Fixado,0,0.1,0.04,0,0
4,KING - BRL,Renda Fixa Brasil Inflação,0,0.15,0.08,0.12,0.109807
5,KING - BRL,Renda Fixa Internacional Cash Equivalent,0,0.05,0,0.005,0.004163


In [14]:
dfPolitica2.rename(columns={'%':'Estratégico'}, inplace=True)
dfPolitica2.head()

,Alocação Estratégica por Mercado,Estratégico
23,Renda Fixa,0.315
24,Retorno Absoluto,0.15
25,Renda Variável,0.225
26,Private Equity,0.255
27,Real Estate,0.055


### - Base de Dados Fundos: (retirar?)

In [15]:
dfDatabase1.head()

,ClassName,ClassLevelTwoName,ClassLevelThreeName,Name,InvestmentTypeName,RedemptionQuotationDelayDayCountName,RedemptionQuotationDelayDayCountValue,ExternalCode,ExternalCodeSource
1,Retorno Absoluto,Retorno Absoluto Brasil,Retorno Absoluto Brasil Macro,Ibiuna Hedge STH FIC FIM,InvestmentFund,Dias Corridos,30,IBIUNA HEDGE STH FIC FIM,Lote 45
2,Renda Variável,Renda Variável Doméstico,Renda Variável Doméstico Ações,Bogari Value JC FIC FIA,InvestmentFund,Dias Corridos,30,BOGARI VALE JC FIC FIM,Lote 45
3,Renda Fixa,Renda Fixa Brasil CDI,RF BR CDI Soberano,BTG Pactual Tesouro Selic FI RF REF DI,InvestmentFund,Dias Corridos,0,TESOURO SELIC FI RF,Lote 45
4,Retorno Absoluto,Retorno Absoluto Brasil,Retorno Absoluto Brasil Macro,Kapitalo Zeta FIQ FIM,InvestmentFund,Dias Úteis,60,KAPITALO FIQ ZETA,Lote 45
5,Retorno Absoluto,Retorno Absoluto Brasil,Retorno Absoluto Brasil Equity Hedge,Moat Capital Equity Hedge FIC FIM,InvestmentFund,Dias Úteis,0,MOAT CAPITAL EQUITY HEDGE FIC FIM,Lote 45


### - Base de Dados e Histórico Benchmark:

In [16]:
dfBenchmarks1.head()

,Classes,Benchmark,Ticker Bloomberg
1,Renda Fixa Brasil CDI,CDI,BZACCETP Index
2,Renda Fixa Brasil Crédito Pós-Fixado,IDA-DI,IDADDI Index
3,Renda Fixa Brasil Pré-Fixado,IRF-M 1+,BZRFIR1+ Index
4,Renda Fixa Brasil Inflação,IMA-B,BZRFIMAB INDEX
5,Renda Fixa Internacional Cash Equivalent,ICE U.S. Treasury Short Bond,IDCOTSTR Index


In [17]:
dfRetornos1.head()

,Data,Renda Fixa Brasil CDI,Renda Fixa Brasil Crédito Pós-Fixado,Renda Fixa Brasil Pré-Fixado,Renda Fixa Brasil Inflação,Renda Fixa Internacional Cash Equivalent,Renda Fixa Internacional Pré-Fixado,Renda Fixa Internacional Inflação,Renda Fixa Internacional Crédito,Retorno Absoluto Brasil,Retorno Absoluto Internacional,Renda Variável Brasil,Renda Variável Internacional,Private Equity Brasil,Private Equity Internacional,Real Estate Brasil,Real Estate Internacional,Commodities
1,2012-01-31 00:00:00,0.008874,0.009366,0.014172,0.016315,0.000059,0.009881,0.023482,0.035732,0.016157,0.027792,0.111326,0.050183,0.08251,0,0.030333,0.083108,0.022312
2,2012-02-29 00:00:00,0.007417,0.008443,0.012284,0.021733,-0.000063,0.003615,-0.003757,0.030352,0.015766,0.020579,0.043433,0.048849,0.094339,0,0.046483,0.039845,0.060629
3,2012-03-30 00:00:00,0.00811,0.009322,0.012667,0.019131,0.000087,-0.001447,-0.011348,0.00147,0.012393,-0.001776,-0.019765,0.012856,0.018434,0.0633,0.040653,0.00741,-0.023503
4,2012-04-30 00:00:00,0.007029,0.00821,0.022992,0.044201,0.000153,0.006232,0.021172,0.008794,0.018731,-0.005121,-0.041709,-0.011356,-0.002949,0,0.014343,0.021775,-0.005123
5,2012-05-31 00:00:00,0.007339,0.008355,0.019004,0.01966,0.00019,0.01041,0.018462,-0.02004,0.008606,-0.026106,-0.118567,-0.086324,-0.110974,0,0.034245,-0.067054,-0.129788


## Rebalanceamento do Portfólio:

### Dataframes Importantes para Análise

Dataframe para checar o Rebalanceamento

In [18]:
book = 'Book'
min = 'Mínimo'
max = 'Máximo'
tat = 'Tático'
atual = 'Atual'

dfRebalance = dfPolitica1[[book, min, max, tat, atual]]
dfRebalance.insert(4, 'Offset', dfRebalance['Atual'] - dfRebalance['Tático'])
dfRebalance.set_index('Book', inplace=True)
dfRebalance

,Mínimo,Máximo,Tático,Offset,Atual
Book,,,,,
Renda Fixa Brasil CDI,0.05,0.35,0.13,0.103557,0.233557
Renda Fixa Brasil Crédito Pós-Fixado,0,0.2,0.09,0.010853,0.100853
Renda Fixa Brasil Pré-Fixado,0,0.1,0,0,0
Renda Fixa Brasil Inflação,0,0.15,0.12,-0.010193,0.109807
Renda Fixa Internacional Cash Equivalent,0,0.05,0.005,-0.000837,0.004163
Renda Fixa Internacional Pré-Fixado,0,0.05,0.01,-0.001674,0.008326
Renda Fixa Internacional Inflação,0,0.05,0.005,-0.002919,0.002081
Renda Fixa Internacional Crédito,0,0.1,0.03,-0.013972,0.016028
Retorno Absoluto Brasil,0,0.2,0.1,-0.012327,0.087673


Dataframe de Análise do Portfólio

In [19]:
resultPortfolio = dfPolitica1[['Book', 'Tático', 'Atual']]
resultPortfolio['Offset'] = resultPortfolio['Atual'] - resultPortfolio['Tático']

resultPortfolio = resultPortfolio[['Book', 'Tático', 'Atual', 'Offset']]
resultPortfolio.set_index('Book', inplace=True)

resultPortfolio

,Tático,Atual,Offset
Book,,,
Renda Fixa Brasil CDI,0.13,0.233557,0.103557
Renda Fixa Brasil Crédito Pós-Fixado,0.09,0.100853,0.010853
Renda Fixa Brasil Pré-Fixado,0,0,0
Renda Fixa Brasil Inflação,0.12,0.109807,-0.010193
Renda Fixa Internacional Cash Equivalent,0.005,0.004163,-0.000837
Renda Fixa Internacional Pré-Fixado,0.01,0.008326,-0.001674
Renda Fixa Internacional Inflação,0.005,0.002081,-0.002919
Renda Fixa Internacional Crédito,0.03,0.016028,-0.013972
Retorno Absoluto Brasil,0.1,0.087673,-0.012327


### TE - Tracking Error do Portfólio:

In [20]:
retTatico = dfRetornos1.copy()
retTatico.set_index('Data', inplace=True)

for index, row in resultPortfolio.iterrows():
    book = index
    currentPerc = row['Tático']

    retTatico[book] = retTatico[book] * currentPerc

retTatico['Retorno Tático'] = retTatico.sum(axis=1)

# retTatico.head()

In [21]:
retAtual = dfRetornos1.copy()
retAtual.set_index('Data', inplace=True)

for index, row in resultPortfolio.iterrows():
    book = index
    currentPerc = row['Atual']

    retAtual[book] = retAtual[book] * currentPerc

retAtual['Retorno Atual'] = retAtual.sum(axis=1)
    
# retAtual.head()

In [22]:
trackingError = pd.DataFrame({
    'Retorno Atual': retAtual['Retorno Atual'],
    'Retorno Tático': retTatico['Retorno Tático']
})

trackingError['Excesso'] = trackingError['Retorno Atual'] - trackingError['Retorno Tático']

trackingError.head()

,Retorno Atual,Retorno Tático,Excesso
Data,,,
2012-01-31,0.036830,0.031728,0.005102
2012-02-29,0.026087,0.020793,0.005294
2012-03-30,0.011634,0.018369,-0.006735
2012-04-30,0.002321,0.003070,-0.000749
2012-05-31,-0.028098,-0.023488,-0.004610


In [23]:
te = trackingError['Excesso'].std()
te = te * math.sqrt(12)

print(f"Tracking Error do Portfólio: {te:.4%}")

def send_notification(title, message):
    notification.notify(
        title=title,
        message=message,
        timeout=10  # Notification will disappear after 10 seconds
    )

if te*100 > perfilRisco:
    surplus = te-perfilRisco/100
    send_notification('Alerta!', f'Você está {surplus:.3%} acima do Perfil de Risco')
else:
    send_notification('Tudo Certo!', f'O Tracking Error está abaixo de {perfilRisco}%')


Tracking Error do Portfólio: 1.6686%


### CTR - Contribution To Risk:

In [24]:
tradingDeskList = dfPolitica1['Book'].to_list()

def listFilterWords(columnList):
    columnList = [item.replace('-', ' ') for item in columnList]
    return [unidecode(item).lower() for item in columnList]

tradingDeskList = listFilterWords(tradingDeskList)


In [25]:
matrizCov = dfRetornos1.copy()

def dfFilterWords(column_name):
    return unidecode(column_name).replace('-', ' ').lower()

matrizCov = matrizCov.rename(columns=dfFilterWords)
matrizCov.reset_index(drop=True)
matrizCov.drop('data', axis=1, inplace=True)
matrizCov = matrizCov.astype(float)
matrizCov = matrizCov[tradingDeskList]


In [26]:
mCov = matrizCov.cov()

mCov

,renda fixa brasil cdi,renda fixa brasil credito pos fixado,renda fixa brasil pre fixado,renda fixa brasil inflacao,renda fixa internacional cash equivalent,renda fixa internacional pre fixado,renda fixa internacional inflacao,renda fixa internacional credito,retorno absoluto brasil,retorno absoluto internacional,renda variavel brasil,renda variavel internacional,private equity brasil,private equity internacional,real estate brasil,real estate internacional,commodities
renda fixa brasil cdi,8.814434e-06,8.619501e-06,0.000011,6.674533e-06,4.353958e-07,-0.000002,-6.421802e-06,-0.000002,0.000006,-0.000009,-0.000002,-0.000015,-0.000010,-0.000019,0.000010,-0.000011,-0.000027
renda fixa brasil credito pos fixado,8.619501e-06,3.496188e-05,0.000013,3.744366e-05,-1.716438e-06,0.000001,-6.401599e-07,0.000054,0.000035,0.000034,0.000135,0.000043,0.000151,0.000029,0.000080,0.000075,0.000114
renda fixa brasil pre fixado,1.062777e-05,1.343194e-05,0.000207,2.418824e-04,2.396870e-06,0.000034,5.910725e-05,0.000096,0.000068,0.000021,0.000431,0.000064,0.000480,0.000023,0.000220,0.000116,-0.000007
renda fixa brasil inflacao,6.674533e-06,3.744366e-05,0.000242,4.394471e-04,-3.761120e-07,0.000051,1.000431e-04,0.000197,0.000131,0.000102,0.000776,0.000208,0.000850,0.000067,0.000347,0.000359,0.000235
renda fixa internacional cash equivalent,4.353958e-07,-1.716438e-06,0.000002,-3.761120e-07,1.543532e-06,0.000001,1.005218e-06,-0.000003,-0.000004,-0.000004,-0.000007,-0.000006,-0.000008,-0.000005,-0.000004,-0.000010,-0.000010
renda fixa internacional pre fixado,-1.621794e-06,1.335374e-06,0.000034,5.108149e-05,1.228657e-06,0.000102,1.292867e-04,0.000103,0.000001,0.000032,0.000088,0.000133,0.000115,0.000014,0.000005,0.000227,-0.000106
renda fixa internacional inflacao,-6.421802e-06,-6.401599e-07,0.000059,1.000431e-04,1.005218e-06,0.000129,2.379075e-04,0.000167,0.000003,0.000078,0.000223,0.000254,0.000233,0.000025,0.000052,0.000397,0.000031
renda fixa internacional credito,-2.387879e-06,5.422794e-05,0.000096,1.974998e-04,-3.082405e-06,0.000103,1.667479e-04,0.000464,0.000123,0.000297,0.000923,0.000724,0.001008,0.000140,0.000351,0.000762,0.000718
retorno absoluto brasil,5.799491e-06,3.510765e-05,0.000068,1.307119e-04,-4.478402e-06,0.000001,2.698196e-06,0.000123,0.000151,0.000102,0.000448,0.000173,0.000545,0.000054,0.000183,0.000197,0.000225
retorno absoluto internacional,-8.759983e-06,3.357756e-05,0.000021,1.024653e-04,-4.160415e-06,0.000032,7.779472e-05,0.000297,0.000102,0.000279,0.000649,0.000598,0.000719,0.000130,0.000238,0.000573,0.000621


In [27]:
pWeights = resultPortfolio["Atual"].to_list()

pVariance = np.dot(pWeights, np.dot(mCov, pWeights))
pVolatility = math.sqrt(pVariance)

print(f"Variância do Portfólio: {pVariance:.5e}")
print(f"Volatilidade do Portfólio: {pVolatility:.5%}")

Variância do Portfólio: 4.58810e-04
Volatilidade do Portfólio: 2.14198%


In [28]:
covPonderada = np.dot(pWeights, mCov)

# covPonderada

In [29]:
riskContribuition = []

for i in range(len(covPonderada)):
    riskContribuition.append(pWeights[i] * covPonderada[i]/pVariance)

riskContribuition = list(i * 100 for i in riskContribuition)

CTR = pd.DataFrame({
    'Ativos': resultPortfolio.index.to_list(),
    'Contribuição [%]': riskContribuition
})

CTR = CTR.sort_values(by='Contribuição [%]', ascending=False)

def highlight_row(s):
    return ['background-color: yellow' if i < nAplicacoes else '' for i in range(len(s))]

highlightedCTR = CTR.style.apply(highlight_row)

highlightedCTR

,Ativos,Contribuição [%]
10,Renda Variável Brasil,46.652426
12,Private Equity Brasil,23.737697
3,Renda Fixa Brasil Inflação,7.082032
11,Renda Variável Internacional,6.316326
13,Private Equity Internacional,4.217852
8,Retorno Absoluto Brasil,3.320610
15,Real Estate Internacional,2.684843
14,Real Estate Brasil,2.625618
1,Renda Fixa Brasil Crédito Pós-Fixado,1.235980
7,Renda Fixa Internacional Crédito,1.233460


### Offset:

In [30]:
dfOffset = resultPortfolio.copy()

dfOffset = dfOffset.iloc[dfOffset['Offset'].abs().argsort()[::-1]]

def highlight_row(s):
    return ['background-color: yellow' if i < nAplicacoes else '' for i in range(len(s))]

highlightedOffset = dfOffset.style.apply(highlight_row)

highlightedOffset

,Tático,Atual,Offset
Book,,,
Private Equity Internacional,0.225000,0.088986,-0.136014
Renda Fixa Brasil CDI,0.130000,0.233557,0.103557
Private Equity Brasil,0.030000,0.077750,0.047750
Real Estate Brasil,0.010000,0.025790,0.015790
Renda Fixa Internacional Crédito,0.030000,0.016028,-0.013972
Renda Variável Brasil,0.150000,0.163536,0.013536
Retorno Absoluto Brasil,0.100000,0.087673,-0.012327
Renda Fixa Brasil Crédito Pós-Fixado,0.090000,0.100853,0.010853
Renda Fixa Brasil Inflação,0.120000,0.109807,-0.010193


### Otimização:

#### Otimização Setup

Preparando o Dataframe de Otimização:

In [31]:
dfOtimizado = dfRebalance.copy()

dfOtimizado['Otimizado CTR'] = dfOtimizado['Atual']
dfOtimizado['Otimizado Offset'] = dfOtimizado['Atual']

dfOtimizado

,Mínimo,Máximo,Tático,Offset,Atual,Otimizado CTR,Otimizado Offset
Book,,,,,,,
Renda Fixa Brasil CDI,0.05,0.35,0.13,0.103557,0.233557,0.233557,0.233557
Renda Fixa Brasil Crédito Pós-Fixado,0,0.2,0.09,0.010853,0.100853,0.100853,0.100853
Renda Fixa Brasil Pré-Fixado,0,0.1,0,0,0,0,0
Renda Fixa Brasil Inflação,0,0.15,0.12,-0.010193,0.109807,0.109807,0.109807
Renda Fixa Internacional Cash Equivalent,0,0.05,0.005,-0.000837,0.004163,0.004163,0.004163
Renda Fixa Internacional Pré-Fixado,0,0.05,0.01,-0.001674,0.008326,0.008326,0.008326
Renda Fixa Internacional Inflação,0,0.05,0.005,-0.002919,0.002081,0.002081,0.002081
Renda Fixa Internacional Crédito,0,0.1,0.03,-0.013972,0.016028,0.016028,0.016028
Retorno Absoluto Brasil,0,0.2,0.1,-0.012327,0.087673,0.087673,0.087673


#### Otimização (CTR):

Pegando as Aplicações que não queremos mudar, baseando-se no CTR

In [32]:
nFilterCTR = CTR['Ativos'].iloc[:nAplicacoes].values
nAssetsCTR = CTR[~CTR['Ativos'].isin(nFilterCTR)]['Ativos'].values

dfOtimizado.insert(6, 'Mudança CTR', dfOtimizado.index.isin(nFilterCTR).astype(int))

Achando Pesos Ótimos do CTR

In [33]:
# Função Objetivo Tracking Error (Minimizar)
def trackingError(w):
    dfOtimizado['Otimizado CTR'] = w
    
    retTatico = dfRetornos1.copy()
    retTatico.set_index('Data', inplace=True)

    for index, row in dfOtimizado.iterrows():
        book = index
        currentPerc = row['Tático']
    
        retTatico[book] = retTatico[book] * currentPerc
    
    retTatico['Retorno Tático'] = retTatico.sum(axis=1)

    retAtual = dfRetornos1.copy()
    retAtual.set_index('Data', inplace=True)

    for index, row in dfOtimizado.iterrows():
        book = index
        currentPerc = row['Otimizado CTR']
    
        retAtual[book] = retAtual[book] * currentPerc
    
    retAtual['Retorno Atual'] = retAtual.sum(axis=1)

    trackingError = pd.DataFrame({
        'Retorno Atual': retAtual['Retorno Atual'],
        'Retorno Tático': retTatico['Retorno Tático']
    })
    
    trackingError['Excesso'] = trackingError['Retorno Atual'] - trackingError['Retorno Tático']

    te = trackingError['Excesso'].std()
    te = te * math.sqrt(12)
    
    return te


# Estimativas Iniciais:
initialGuess = np.array(dfOtimizado['Otimizado CTR'])


# Limites
paramRanges = [(min_val, max_val) for min_val, max_val in zip(dfOtimizado['Mínimo'], dfOtimizado['Máximo'])]


# Restrições
cons = [
    {'type': 'eq', 'fun': lambda w: np.sum(w) - 1.0}
]


# Solução:
def solve(num_fixed_params, initialGuess, paramRanges, cons):
    for i in num_fixed_params:
        paramRanges[i] = (initialGuess[i], initialGuess[i])

    result = minimize(trackingError, initialGuess, bounds=paramRanges, constraints=cons, options={'disp': True})
    return result

values_to_findCTR = nAssetsCTR
num_fixed_paramsCTR = [dfOtimizado.index.get_loc(name) for name in values_to_findCTR]

resultCTR = solve(num_fixed_paramsCTR, initialGuess, paramRanges, cons)


# Resultados
wOptimalCTR = resultCTR.x
teOptimalCTR = resultCTR.fun

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.013713790241862608
            Iterations: 4
            Function evaluations: 20
            Gradient evaluations: 4


In [34]:
print(f'Novo Tracking Error: {teOptimalCTR:.4%}')

Novo Tracking Error: 1.3714%


In [35]:
dfOtimizado

,Mínimo,Máximo,Tático,Offset,Atual,Otimizado CTR,Mudança CTR,Otimizado Offset
Book,,,,,,,,
Renda Fixa Brasil CDI,0.05,0.35,0.13,0.103557,0.233557,0.233557,0,0.233557
Renda Fixa Brasil Crédito Pós-Fixado,0,0.2,0.09,0.010853,0.100853,0.100853,0,0.100853
Renda Fixa Brasil Pré-Fixado,0,0.1,0,0,0,0.000000,0,0
Renda Fixa Brasil Inflação,0,0.15,0.12,-0.010193,0.109807,0.136284,1,0.109807
Renda Fixa Internacional Cash Equivalent,0,0.05,0.005,-0.000837,0.004163,0.004163,0,0.004163
Renda Fixa Internacional Pré-Fixado,0,0.05,0.01,-0.001674,0.008326,0.008326,0,0.008326
Renda Fixa Internacional Inflação,0,0.05,0.005,-0.002919,0.002081,0.002081,0,0.002081
Renda Fixa Internacional Crédito,0,0.1,0.03,-0.013972,0.016028,0.016028,0,0.016028
Retorno Absoluto Brasil,0,0.2,0.1,-0.012327,0.087673,0.087673,0,0.087673


#### Otimização (Offset):

Pegando as Aplicações que não queremos mudar, baseando-se no Offset

In [36]:
nFilterOffset = dfOffset['Offset'].iloc[:nAplicacoes].values
nAssetsOffset = dfOffset[~dfOffset['Offset'].isin(nFilterOffset)]['Offset'].index

dfOtimizado['Mudança Offset'] = np.where(dfOtimizado['Offset'].isin(nFilterOffset), 1, 0)


Achando Pesos Ótimos do Offset

In [37]:
# Função Objetivo Tracking Error (Minimizar)
def trackingError(w):
    dfOtimizado['Otimizado Offset'] = w
    
    retTatico = dfRetornos1.copy()
    retTatico.set_index('Data', inplace=True)

    for index, row in dfOtimizado.iterrows():
        book = index
        currentPerc = row['Tático']
        # print(book, currentPerc, retTatico[book])
    
        retTatico[book] = retTatico[book] * currentPerc
    
    retTatico['Retorno Tático'] = retTatico.sum(axis=1)

    retAtual = dfRetornos1.copy()
    retAtual.set_index('Data', inplace=True)

    for index, row in dfOtimizado.iterrows():
        book = index
        currentPerc = row['Otimizado Offset']
    
        retAtual[book] = retAtual[book] * currentPerc
    
    retAtual['Retorno Atual'] = retAtual.sum(axis=1)

    trackingError = pd.DataFrame({
        'Retorno Atual': retAtual['Retorno Atual'],
        'Retorno Tático': retTatico['Retorno Tático']
    })
    
    trackingError['Excesso'] = trackingError['Retorno Atual'] - trackingError['Retorno Tático']

    te = trackingError['Excesso'].std()
    te = te * math.sqrt(12)
    
    return te


# Estimativas Iniciais:
initialGuess = np.array(dfOtimizado['Otimizado Offset'])


# Limites
paramRanges = [(min_val, max_val) for min_val, max_val in zip(dfOtimizado['Mínimo'], dfOtimizado['Máximo'])]


# Restrições
cons = [
    {'type': 'eq', 'fun': lambda w: np.sum(w) - 1.0}
]


# Solução:
def solve(num_fixed_params, initialGuess, paramRanges, cons):
    for i in num_fixed_params:
        paramRanges[i] = (initialGuess[i], initialGuess[i])

    result = minimize(trackingError, initialGuess, bounds=paramRanges, constraints=cons, options={'disp': True})
    return result

values_to_findOffset = nAssetsOffset
num_fixed_paramsOffset = [dfOtimizado.index.get_loc(name) for name in values_to_findOffset]

resultOffset = solve(num_fixed_paramsOffset, initialGuess, paramRanges, cons)


# Resultados
wOptimalOffset = resultOffset.x
teOptimalOffset = resultOffset.fun

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.002461164524020437
            Iterations: 7
            Function evaluations: 39
            Gradient evaluations: 7


In [38]:
print(f'Novo Tracking Error: {teOptimalOffset:.4%}')

Novo Tracking Error: 0.2461%


In [39]:
dfOtimizado

,Mínimo,Máximo,Tático,Offset,Atual,Otimizado CTR,Mudança CTR,Otimizado Offset,Mudança Offset
Book,,,,,,,,,
Renda Fixa Brasil CDI,0.05,0.35,0.13,0.103557,0.233557,0.233557,0,0.156645,1
Renda Fixa Brasil Crédito Pós-Fixado,0,0.2,0.09,0.010853,0.100853,0.100853,0,0.100853,0
Renda Fixa Brasil Pré-Fixado,0,0.1,0,0,0,0.000000,0,0.000000,0
Renda Fixa Brasil Inflação,0,0.15,0.12,-0.010193,0.109807,0.136284,1,0.109807,0
Renda Fixa Internacional Cash Equivalent,0,0.05,0.005,-0.000837,0.004163,0.004163,0,0.004163,0
Renda Fixa Internacional Pré-Fixado,0,0.05,0.01,-0.001674,0.008326,0.008326,0,0.008326,0
Renda Fixa Internacional Inflação,0,0.05,0.005,-0.002919,0.002081,0.002081,0,0.002081,0
Renda Fixa Internacional Crédito,0,0.1,0.03,-0.013972,0.016028,0.016028,0,0.016028,0
Retorno Absoluto Brasil,0,0.2,0.1,-0.012327,0.087673,0.087673,0,0.087673,0


## Resultados

Tabela de Resultados do Rebalanceamento

In [40]:
dfResultados = dfOtimizado.copy()

otimizacoes = ['CTR', 'Offset']

for otimizacao in otimizacoes:
    acao = []
    for index, row in dfResultados.iterrows():
        text = ''
        percDiff = row[f'Otimizado {otimizacao}'] - row['Atual']
        newValue = abs(percDiff) * valorCarteira
        assetValue = locale.format_string("%.3f", newValue, grouping=True, monetary=True)
        
        if percDiff < 0:
            text += f'Vender {assetValue}'
    
        elif percDiff > 0:
            text += f'Comprar {assetValue}'
    
        else:
            text += '-'
    
        acao.append(text)

    dfResultados[f'Ação {otimizacao}'] = acao

dfResultados = dfResultados[['Mínimo', 'Máximo', 'Tático', 'Offset', 'Atual', 'Otimizado CTR', 'Mudança CTR', 'Ação CTR', 'Otimizado Offset', 'Mudança Offset', 'Ação Offset']]

dfResultados

,Mínimo,Máximo,Tático,Offset,Atual,Otimizado CTR,Mudança CTR,Ação CTR,Otimizado Offset,Mudança Offset,Ação Offset
Book,,,,,,,,,,,
Renda Fixa Brasil CDI,0.05,0.35,0.13,0.103557,0.233557,0.233557,0,-,0.156645,1,"Vender 945.697,525"
Renda Fixa Brasil Crédito Pós-Fixado,0,0.2,0.09,0.010853,0.100853,0.100853,0,-,0.100853,0,-
Renda Fixa Brasil Pré-Fixado,0,0.1,0,0,0,0.000000,0,-,0.000000,0,-
Renda Fixa Brasil Inflação,0,0.15,0.12,-0.010193,0.109807,0.136284,1,"Comprar 325.560,929",0.109807,0,-
Renda Fixa Internacional Cash Equivalent,0,0.05,0.005,-0.000837,0.004163,0.004163,0,-,0.004163,0,-
Renda Fixa Internacional Pré-Fixado,0,0.05,0.01,-0.001674,0.008326,0.008326,0,-,0.008326,0,-
Renda Fixa Internacional Inflação,0,0.05,0.005,-0.002919,0.002081,0.002081,0,-,0.002081,0,-
Renda Fixa Internacional Crédito,0,0.1,0.03,-0.013972,0.016028,0.016028,0,-,0.016028,0,-
Retorno Absoluto Brasil,0,0.2,0.1,-0.012327,0.087673,0.087673,0,-,0.087673,0,-


Tabela dos Tracking Errors

In [41]:
data = {
    '': ['Tracking Error[%]'],
    'Atual': [te*100],
    'CTR': [teOptimalCTR*100],
    'Offset': [teOptimalOffset*100]
}

dfTableTE = pd.DataFrame(data)
dfTableTE.set_index('', inplace=True)

dfTableTE

,Atual,CTR,Offset
,,,
Tracking Error[%],1.668571,1.371379,0.246116


Convertendo um Excel de Output

In [42]:

def send_notification(title, message):
    notification.notify(
        title=title,
        message=message,
        timeout=10  # Notification will disappear after 10 seconds
    )

excelName = portfolioCliente.split('.')[0]

if os.path.exists(f'{PATH}/Resultados'):
    pass
else:
    os.mkdir(f'{PATH}/Resultados')

excel_file_path = f'{PATH}/Resultados/Resultados ({excelName}).xlsx'

# Send Notification Status
try:
    with pd.ExcelWriter(excel_file_path, engine='xlsxwriter') as writer:
        dfResultados.to_excel(writer, sheet_name='Rebalanceamento')
        dfTableTE.to_excel(writer, sheet_name='Tracking Error')
    
    send_notification("Success!", f"Excel File ({excelName}) Created")

except:
    send_notification("Warning!", "Feche o Excel Antes de Rodar o Código")

<br>

---

In [43]:
send_notification("Warning!", "Feche o Excel Antes de Rodar o Código")

<br>